This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [221]:
# First, import the relevant modules
import requests
import json
from collections import Counter

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [222]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'
start_date = '2019-03-01'
end_date = '2019-03-01'
url = f'https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}'
r = requests.get(url + f'?api_key={API_KEY}&start_date={start_date}&end_date={end_date}')
json_data = r.json()

In [223]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
for k, v in json_data.items():
    print(k,':',type(v))
    
    for k2, v2 in json_data[k].items():
        print('\t',k2,':',type(v2))

dataset_data : <class 'dict'>
	 limit : <class 'NoneType'>
	 transform : <class 'NoneType'>
	 column_index : <class 'NoneType'>
	 column_names : <class 'list'>
	 start_date : <class 'str'>
	 end_date : <class 'str'>
	 frequency : <class 'str'>
	 data : <class 'list'>
	 collapse : <class 'NoneType'>
	 order : <class 'NoneType'>


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [224]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
#    (keep in mind that the date format is YYYY-MM-DD).

# define parameters to query
database_code = 'FSE'
dataset_code = 'AFX_X'
return_format = 'json'
start_date = '2017-01-01'
end_date = '2017-12-31'
url = f'https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}'
params = f'?api_key={API_KEY}&start_date={start_date}&end_date={end_date}'
# collect API response
r = requests.get(url + params)

In [225]:
# 2. Convert the returned JSON object into a Python dictionary.
dict_data = r.json()

In [226]:
# Preview column names within dataset
print(dict_data['dataset_data']['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [227]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

# Get index of 'Open' column within data
open_index = dict_data['dataset_data']['column_names'].index('Open')

# Test for highest and lowest opening prices
high_low_prices_initialized = False
for ind, row in enumerate(dict_data['dataset_data']['data']):
    
    # initialize highest and lowest opening prices
    if row[open_index] is not None:
        if not high_low_prices_initialized:
            highest_opening_price = row[open_index]
            lowest_opening_price = row[open_index]
            high_low_prices_initialized = True
        
        # test for lowest price
        if row[open_index] < lowest_opening_price:
            lowest_opening_price = row[open_index]

        # test for highest price
        if row[open_index] > highest_opening_price:
            highest_opening_price = row[open_index]

print('The lowest opening price for the stock in this period was $' + str(lowest_opening_price))
print('The highest opening price for the stock in this period was $' + str(highest_opening_price))

The lowest opening price for the stock in this period was $34.0
The highest opening price for the stock in this period was $53.11


In [228]:
# 4. What was the largest change in any one day (based on High and Low price)?

# Get indices of 'High' and 'Low' columns within data
high_index = dict_data['dataset_data']['column_names'].index('High')
low_index = dict_data['dataset_data']['column_names'].index('Low')

# Test for largest daily change based on high and low prices
largest_daily_change = 0.0
for ind, row in enumerate(dict_data['dataset_data']['data']):
    
    # initialize highest and lowest opening prices
    if row[open_index] is not None:
        
        # test for largest daily change
        if (row[high_index] - row[low_index]) > largest_daily_change:
            largest_daily_change = row[high_index] - row[low_index]

print('The largest change in any one day (based on High and Low price) was $' + str(largest_daily_change))

The largest change in any one day (based on High and Low price) was $2.8100000000000023


In [229]:
# 5. What was the largest change between any two days (based on Closing Price)?

# Get index of 'Close' column within data
close_index = dict_data['dataset_data']['column_names'].index('Close')

# Test for largest change between any two days based on Closing Price
largest_two_day_change = 0.0
for ind, row in enumerate(dict_data['dataset_data']['data']):
    if ind > 0:
        # initialize highest and lowest opening prices
        if (row[close_index] is not None) and (row[close_index - 1] is not None):

            # test for largest daily change
            if (row[close_index] - row[close_index - 1]) > largest_two_day_change:
                largest_two_day_change = row[close_index] - row[close_index - 1]

print('The largest change between any two days (based on Closing Price) was $' + str(largest_two_day_change))

The largest change between any two days (based on Closing Price) was $1.75


In [230]:
# 6. What was the average daily trading volume during this year?

# Get index of 'Traded Volume' column within data
traded_volume_index = dict_data['dataset_data']['column_names'].index('Traded Volume')

# Record daily traded volume
daily_trade_vol_sum = 0.0
daily_count = 0
for ind, row in enumerate(dict_data['dataset_data']['data']):
    
    # initialize highest and lowest opening prices
    if row[traded_volume_index] is not None:

        # sum daily traded volume
        daily_trade_vol_sum = daily_trade_vol_sum + row[traded_volume_index]
        daily_count = daily_count + 1

average_daily_trade_volume = daily_trade_vol_sum / daily_count

print('The average daily trading volume during the year 2017 was $' + str(average_daily_trade_volume))

The average daily trading volume during the year 2017 was $89124.33725490196


In [231]:
# 7. (Optional) What was the median trading volume during this year.
#    (Note: you may need to implement your own function for calculating the median.)

# Get index of 'Traded Volume' column within data
traded_volume_index = dict_data['dataset_data']['column_names'].index('Traded Volume')

# Record daily traded volume into list
trade_vol_list = []
for ind, row in enumerate(dict_data['dataset_data']['data']):
    
    # initialize highest and lowest opening prices
    if row[traded_volume_index] is not None:

        # record daily traded volume
        trade_vol_list.append(row[traded_volume_index])

# find median
if len(trade_vol_list) % 2 == 0: # even
    median_index_low = len(trad_vol_list) / 2
    median_index_high = median_index_low + 1
    median_traded_volume = (trade_vol_list[median_index_high] - trade_vol_list[median_index_low]) / 2
    print(median_index_low)
elif len(trade_vol_list) % 2 != 0: # odd
    median_index = int(len(trade_vol_list) / 2) + 1
    median_traded_volume = trade_vol_list[median_index]

print('The median trading volume during the year 2017 was $' + str(median_traded_volume))

The median trading volume during the year 2017 was $117165.0
